In [19]:
from esper.stdlib import *
from esper.prelude import *
import os
import struct
import json
from tqdm import tqdm

def decode(pose):
    return pose.keypoints.tobytes()

In [2]:
import scannerpy
import scannertools as st

In [3]:
videos = Video.objects.filter(id=1).all()

In [6]:
db = scannerpy.Database()

In [8]:
frames = []

In [9]:
poses = st.pose_detection.detect_poses(
    db,
    videos=[video.for_scannertools() for video in videos],
    frames=frames
)

In [12]:
for poses_in_frame in poses[0].load():
    if len(poses_in_frame) == 0:
        continue
    pose = poses_in_frame[0]
    break

In [22]:
len(pose.keypoints.tobytes())

1560

In [28]:
pose.keypoints.tobytes()

b"X\x05\x0c?Me\xd2>\xbd\x86c?G&\x0b?Z\x1f\n?89U?kx\xfb>8\x1f\n?%r`?18\xf6>\xbc;=?E\x13:?E\xf9\xf7>R(l?5\x8f(?\xce\xf8\x17?\x8d\x1d\n?\xa0gG?\x1e\x0f\x1c?\xe7\xbd9?\xe7b;?^9\x1d?w\x0ec?\xd5x\r?\xd3\x9a\x05?\x96\xa6Z?\x0f~\x1c?\xf2E\x03?\x96\xc4\x7f?\xb3\x97\xcf=\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11z\x14?\xa4?Y?\x1d\xd9%?kz\x14?,\xc4\x7f?Y$\x93=\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x8ce\t?\xb1\xd4\xc5>\x03mu?\xad\x10\x0e?\x007\xc7>\x8b\x05n?'2\x06?$\x9f\xcf>4\xe89?\xfb\x8e\x11?!\xff\xd0>\x08\xf6U?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [36]:
struct.pack('f', json.loads(FaceFeatures.objects.get(id=1).features.tobytes().decode())[0])

b'\xafO\xd8\xbd'

In [39]:
POSE_DIR = '/app/data/pose/'
IDS_FILE = os.path.join(POSE_DIR, 'pose_ids.bin')
POSE_FILE = os.path.join(POSE_DIR, 'pose_binary.bin')
ENDIAN = 'little'
DIMENSIONS = 1560

if not os.path.exists(POSE_DIR):
    os.makedirs(POSE_DIR)
    
with open(IDS_FILE, 'wb') as f_ids, open(POSE_FILE, 'wb') as f_poses:
    for i, x in tqdm(enumerate([pose])):
        pose_id = i
        pose_binary = decode(pose)
        assert len(pose_binary) == DIMENSIONS, 'Incorrect dimensions: {} != {}'.format(
            len(pose_binary), DIMENSIONS)
        f_ids.write(pose_id.to_bytes(8, byteorder=ENDIAN))
        f_poses.write(pose_binary)
    print('Done! ({} poses written)'.format(i + 1))

1it [00:00, 2269.65it/s]

Done! (1 poses written)


In [8]:
import esper.pose_wrapper as pose_wrapper

In [2]:
from query.models import PoseMeta

In [3]:
poses = PoseMeta.objects.order_by('id').all()

In [4]:
ids = sorted([p.id for p in poses])

In [5]:
ids_exist = pose_wrapper.exists(poses)

In [6]:
non_existant_ids = [pid for pid, exists in zip(ids, ids_exist) if not exists]

In [7]:
len(non_existant_ids)

27869

In [13]:
non_existant_ids[0]

9562306

In [14]:
non_existant_ids[-1]

9590174

In [16]:
PoseMeta.objects.filter(id__in=non_existant_ids).all().delete()

(27869, {'query.PoseMeta': 27869})

In [40]:
pose = PoseMeta.objects.filter(frame__video_id=1).order_by('id').annotate(
    video_id=F('frame__video_id'),
    frame_num=F('frame__number')
).all()[1000]

In [41]:
pose.id

95339

In [42]:
pose_wrapper.get([pose.id])[0][1]

[0.2314654439687729,
 0.23045623302459717,
 0.7422003746032715,
 0.2781543433666229,
 0.38083043694496155,
 0.5075578093528748,
 0.2588018476963043,
 0.36718079447746277,
 0.38940533995628357,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.29068881273269653,
 0.38635513186454773,
 0.6175556182861328,
 0.3020820915699005,
 0.6790328025817871,
 0.27526673674583435,
 0.2588046193122864,
 0.8759776949882507,
 0.12803933024406433,
 0.2542441785335541,
 0.8185346126556396,
 0.1653899848461151,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2849982976913452,
 0.8404226303100586,
 0.24222195148468018,
 0.24741172790527344,
 0.9977094531059265,
 0.05633373558521271,
 0.0,
 0.0,
 0.0,
 0.23033268749713898,
 0.21128970384597778,
 0.05495050922036171,
 0.24058140814304352,
 0.20856748521327972,
 0.7915856242179871,
 0.0,
 0.0,
 0.0,
 0.26790061593055725,
 0.2276647984981537,
 0.7254486680030823,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,

In [28]:
from query.models import Pose
import numpy as np

In [65]:
np.array(pose_wrapper.get([pose.id])[0][1]).tobytes('F')

b'\x00\x00\x00\xe0\xa8\xa0\xcd?\x00\x00\x00\x00\x97\x7f\xcd?\x00\x00\x00\x00\x1b\xc0\xe7?\x00\x00\x00\xe0G\xcd\xd1?\x00\x00\x00\xa0\x86_\xd8?\x00\x00\x00\xe0\xe9=\xe0?\x00\x00\x00\xa05\x90\xd0?\x00\x00\x00\xe0\xe3\x7f\xd7?\x00\x00\x00`\x04\xec\xd8?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\xa5\x9a\xd2?\x00\x00\x00\xe0\n\xba\xd8?\x00\x00\x00\x00\x04\xc3\xe3?\x00\x00\x00 PU\xd3?\x00\x00\x00\x00\xa3\xba\xe5?\x00\x00\x00`\xf8\x9d\xd1?\x00\x00\x00@A\x90\xd0?\x00\x00\x00`\x02\x08\xec?\x00\x00\x00\xc0\x97c\xc0?\x00\x00\x00`\x89E\xd0?\x00\x00\x00\x80o1\xea?\x00\x00\x00\xc0\x7f+\xc5?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80i=\xd2?\x00\x00\x00\x00\xbe\xe4\xea?\x00\x00\x00

In [43]:
np.array(pose_wrapper.get([pose.id])[0][1]).tobytes()

b'\x00\x00\x00\xe0\xa8\xa0\xcd?\x00\x00\x00\x00\x97\x7f\xcd?\x00\x00\x00\x00\x1b\xc0\xe7?\x00\x00\x00\xe0G\xcd\xd1?\x00\x00\x00\xa0\x86_\xd8?\x00\x00\x00\xe0\xe9=\xe0?\x00\x00\x00\xa05\x90\xd0?\x00\x00\x00\xe0\xe3\x7f\xd7?\x00\x00\x00`\x04\xec\xd8?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\xa5\x9a\xd2?\x00\x00\x00\xe0\n\xba\xd8?\x00\x00\x00\x00\x04\xc3\xe3?\x00\x00\x00 PU\xd3?\x00\x00\x00\x00\xa3\xba\xe5?\x00\x00\x00`\xf8\x9d\xd1?\x00\x00\x00@A\x90\xd0?\x00\x00\x00`\x02\x08\xec?\x00\x00\x00\xc0\x97c\xc0?\x00\x00\x00`\x89E\xd0?\x00\x00\x00\x80o1\xea?\x00\x00\x00\xc0\x7f+\xc5?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80i=\xd2?\x00\x00\x00\x00\xbe\xe4\xea?\x00\x00\x00

In [84]:
sample_pose = Pose(
    keypoints=np.array(pose_wrapper.get([pose.id])[0][1]),
    labeler=Labeler.objects.get(name='openpose'))

In [85]:
sample_pose.pose_keypoints()

array([[-3.6893488e+19,  1.6064653e+00,  0.0000000e+00],
       [ 1.6054562e+00,  0.0000000e+00,  1.8105501e+00],
       [-3.6893488e+19,  1.6390771e+00, -1.0842022e-19],
       [ 1.6904151e+00, -3.6893488e+19,  1.7518893e+00],
       [-1.0842022e-19,  1.6294008e+00, -3.6893488e+19],
       [ 1.6835903e+00,  3.6893488e+19,  1.6947026e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 2.0000000e+00,  1.6453444e+00, -3.6893488e+19],
       [ 1.6931775e+00,  0.0000000e+00,  1.7793889e+00],
       [ 1.0842022e-19,  1.6510410e+00,  0.0000000e+00],
       [ 1.7947582e+00,  3.6893488e+19,  1.6376333e+00],
       [ 2.0000000e+00,  1.6294023e+00,  3.6893488e+19],
       [ 1.8439944e+00, -2.0000000e+00,  1.5030392e+00],
       [ 3.6893488e+19,  1.6271220e+00, -0.0000000e+00],
       [ 1.8296336e+00, -2.0000

In [34]:
from esper.stdlib import *

In [78]:
class MyPose():
    def __init__(self, keypoints, pose_id, labeler):
        self.kp = np.array(keypoints).reshape(130, 3)
        self.id = pose_id
        self.labeler = labeler
    
    POSE_KEYPOINTS = 18
    FACE_KEYPOINTS = 70
    HAND_KEYPOINTS = 21

    Nose = 0
    Neck = 1
    RShoulder = 2
    RElbow = 3
    RWrist = 4
    LShoulder = 5
    LElbow = 6
    LWrist = 7
    RHip = 8
    RKnee = 9
    RAnkle = 10
    LHip = 11
    LKnee = 12
    LAnkle = 13
    REye = 14
    LEye = 15
    REar = 16
    LEar = 17
    Background = 18

    def pose_keypoints(self):
        kp = self.kp
        return kp[:self.POSE_KEYPOINTS, :]

    def face_keypoints(self):
        kp = self.kp
        return kp[self.POSE_KEYPOINTS:(self.POSE_KEYPOINTS + self.FACE_KEYPOINTS), :]

    def hand_keypoints(self):
        kp = self.kp
        base = kp[self.POSE_KEYPOINTS + self.FACE_KEYPOINTS:, :]
        return [base[:self.HAND_KEYPOINTS, :], base[self.HAND_KEYPOINTS:, :]]

In [79]:
sample_pose_mine = MyPose(np.array(pose_wrapper.get([pose.id])[0][1]), pose.id, pose.labeler)

In [80]:
sample_pose_dict = pose_to_dict(sample_pose_mine)

In [81]:
materialized_result = [{
    'video': pose.video_id,
    'min_frame': pose.frame_num,
    'objects': [sample_pose_dict]
}]

In [82]:
from esper.prelude import *

In [83]:
esper_widget(simple_result(materialized_result, 'pose'))

VGridWidget(jsglobals={'queries': [['All faces', 'def all_faces():\n    from query.models import Face\n    fro…

In [86]:
pose_ids = [p.id for p in PoseMeta.objects.filter(frame__video_id=1, frame__number=9792).all()]

In [87]:
poses = pose_wrapper.get(pose_ids)

In [89]:
materialized_result = [{
    'video': 1,
    'min_frame': 9792,
    'objects':[
        pose_to_dict(
            MyPose(np.array(p[1]), pose_ids[i], Labeler.objects.get(name='openpose'))
        ) for i, p in enumerate(poses)
    ]
}]

In [90]:
esper_widget(simple_result(materialized_result, 'pose'))

VGridWidget(jsglobals={'queries': [['All faces', 'def all_faces():\n    from query.models import Face\n    fro…

In [101]:
import esper.pose as pose_wrapper

In [103]:
poses = pose_wrapper.get(PoseMeta.objects.filter(frame__video_id=1, frame__number=9792).all())

In [104]:
materialized_result = [{
    'video': 1,
    'min_frame': 9792,
    'objects':[
        pose_to_dict(pose) for pose in poses
    ]
}]

In [105]:
esper_widget(simple_result(materialized_result, 'pose'))

VGridWidget(jsglobals={'queries': [['All faces', 'def all_faces():\n    from query.models import Face\n    fro…

In [113]:
from query.models import PoseMeta
from esper.stdlib import pose_to_dict, simple_result
import esper.pose as pose_wrapper
import rekall as rk
from rekall.video_interval_collection import VideoIntervalCollection
from rekall.parsers import in_array
from rekall.merge_ops import payload_plus
from esper.rekall import intrvllists_to_result_with_objects

# We are recreating the work in qs_to_result to show how to get pose objects
STRIDE = 1000
LIMIT = 100

pose_meta_qs = PoseMeta.objects.annotate(
    min_frame=F('frame__number'),
    max_frame=F('frame__number'),
    video_id=F('frame__video_id'))

# Use coalesce to get a list of frames we want
frames = VideoIntervalCollection.from_django_qs(
    pose_meta_qs[:LIMIT*STRIDE:STRIDE],
    with_payload=lambda pose_meta_obj: pose_meta_obj.video_id
).coalesce()

# pose_wrapper.get returns a list of PoseWrapper objects
poses = frames.map(lambda interval: (
    interval.start, interval.end,
    pose_wrapper.get(pose_meta_qs.filter(
        video_id=interval.payload,
        min_frame=interval.start).all())
))

result = intrvllists_to_result_with_objects(
    poses,
    lambda pose_wrappers, video_id: [pose_to_dict(pw) for pw in pose_wrappers]
)

In [114]:
esper_widget(result)

VGridWidget(jsglobals={'queries': [['All faces', 'def all_faces():\n    from query.models import Face\n    fro…

In [109]:
def test_map(interval):
    qs = pose_meta_qs.filter(
        video_id=interval.payload,
        min_frame=interval.start)
    print(qs)
    return (interval.start, interval.end, pose_wrapper.get(qs))

frames.map(test_map)

<QuerySet [PoseMeta(id: 5045973)]>


AssertionError: 0 != 1

In [2]:
from esper.prelude import *
from esper.shot_scale import ShotScale as ShotScaleEnum, label_videos_with_shot_scale
from query.models import Frame, ShotScale, Face, Pose, VideoTag
from tqdm import tqdm

shot_scale_enum_to_row = {}
for scale in ShotScaleEnum:
    shot_scale_enum_to_row[scale] = ShotScale.objects.get(name=scale.name.lower())
print(shot_scale_enum_to_row)

video_ids = [19]

print(video_ids)

for video_id in tqdm(video_ids):
    frames = label_videos_with_shot_scale([video_id]).get_intervallist(video_id)
    for frame in frames.get_intervals():
        frame_id = frame.payload['frame_id']
        shot_scale = shot_scale_enum_to_row[frame.payload['shot_scale']]
        frame_obj = Frame.objects.get(pk=frame_id)
        frame_obj.shot_scale = shot_scale
        frame_obj.save()


  0%|                                                                                                                                                                                                                  | 0/1 [00:00<?, ?it/s]

{<ShotScale.UNKNOWN: 0>: ShotScale(id: 1, name: unknown), <ShotScale.EXTREME_LONG: 1>: ShotScale(id: 2, name: extreme_long), <ShotScale.LONG: 2>: ShotScale(id: 3, name: long), <ShotScale.MEDIUM_LONG: 3>: ShotScale(id: 4, name: medium_long), <ShotScale.MEDIUM: 4>: ShotScale(id: 5, name: medium), <ShotScale.MEDIUM_CLOSE_UP: 5>: ShotScale(id: 8, name: medium_close_up), <ShotScale.CLOSE_UP: 6>: ShotScale(id: 6, name: close_up), <ShotScale.EXTREME_CLOSE_UP: 7>: ShotScale(id: 7, name: extreme_close_up)}
[19]



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [02:17<00:00, 137.99s/it]
